In [113]:
import pandas as pd
import openpyxl
from tqdm import tqdm
import unidecode
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import warnings
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import math
from operator import itemgetter

import unidecode
import string
import re
from fuzzywuzzy import fuzz
import os

import glob as f

## Dependientes

In [114]:
print("SGP_educaciontot SGP_educacion_ps SGP_educacion_c SGP_educacion_cg SGP_educacion_cm SGP_saludtot inv_en_salud DI_capadmin SGP_pinfancia IGA_atencion_ciud IGA_sisben".split(" "))

['SGP_educaciontot', 'SGP_educacion_ps', 'SGP_educacion_c', 'SGP_educacion_cg', 'SGP_educacion_cm', 'SGP_saludtot', 'inv_en_salud', 'DI_capadmin', 'SGP_pinfancia', 'IGA_atencion_ciud', 'IGA_sisben']


### Panel municipal

In [115]:
bg = pd.read_stata("PANEL_DE_EDUCACION(2021).dta")
bg = bg[['codmpio', 'ano', 'docen_total', 'n_icfes', 'pobsimple_primaria', 'pobsimple_secund', 'ind_alfa', 's11_total']]
bg = bg.rename(columns= {"ano": "date"})
bg['date'] = bg['date'].dt.year
ed = bg.copy()

In [116]:
bg = pd.read_stata("PANEL_CARACTERISTICAS_GENERALES(2022).dta")
bg = bg[['codmpio', 'ano', 'gandina', 'gcaribe', 'gpacifica', 'gorinoquia', 'gamazonia', 
        'nbi', 'pobreza', 'gini', 'altura', 'areaoficialkm2', 'ao_crea', 'coddepto', 
        'indrural', 'discapital', 'pobl_tot']]

bg = bg.rename(columns= {"ano": "date"})
# bg['date'] = bg['date'].dt.year
cg = bg.copy()

In [117]:
bg = pd.read_stata("PANEL_BUEN_GOBIERNO(2022).dta")
bg = bg[['codmpio', 'ano', 'SGP_educaciontot', 'SGP_educacion_ps', 
        'SGP_educacion_c', 'SGP_educacion_cg', 'SGP_educacion_cm', 'SGP_saludtot', 
        'inv_en_salud', 'SGP_pinfancia', 
        'IGA_atencion_ciud', 'IGA_sisben', 
        'DF_ahorro', 'DF_desemp_fisc', 'DI_eficacia', 'DI_eficiencia', 'DI_capadmin', 'DI_desemp_int']]
bg = bg.rename(columns= {"ano": "date"})
# bg['date'] = bg['date'].dt.year
bg = bg.copy()

## Historicas

In [118]:
ac = pd.read_excel("variables_acemoglu.xlsx")
ac.columns = ["codmpio", "dist_creal", "petroleo", "lat", "long", "dmagdalena", "dcauca", "dcosta"]
ac = ac[["codmpio", "dist_creal", "dmagdalena", "dcauca", "dcosta"]]

In [119]:
geo = pd.read_excel("geo_dane.xlsx")

In [120]:
geoh = geo.merge(ac, "left", "codmpio")

In [121]:
# geoh.to_excel("geo_hist.xlsx", index = False)

In [122]:
from scipy.spatial.distance import euclidean

def encontrar_municipio_mas_cercano(codigo1, lista_codigos, data):
    """
    Encuentra el municipio más cercano a un municipio dado (codigo1) de una lista de municipios.
    
    :param codigo1: Código del municipio de referencia.
    :param lista_codigos: Lista de códigos de municipios para comparar.
    :param data: DataFrame con los datos de los municipios.
    :return: Código del municipio más cercano.
    """
    # Extraer latitud y longitud del municipio de referencia
    municipio_ref = data[data['codmpio'] == codigo1]
    if municipio_ref.empty:
        return None  # Si el código1 no existe en los datos
    
    lat_ref, long_ref = municipio_ref.iloc[0]['lat'], municipio_ref.iloc[0]['long']
    
    # Filtrar los municipios de la lista
    municipios_lista = data[data['codmpio'].isin(lista_codigos)]
    
    # Calcular distancia euclidiana
    distancias = municipios_lista.apply(lambda x: euclidean((lat_ref, long_ref), (x['lat'], x['long'])), axis=1)
    
    # Encontrar el índice del mínimo valor de distancia
    min_dist_idx = distancias.idxmin()
    
    # Devolver el código del municipio más cercano
    return municipios_lista.loc[min_dist_idx, 'codmpio'] if not distancias.empty else None

vecinos = []
notnans1 = geoh[geoh.dist_creal.notna()]
nans1 = geoh[geoh.dist_creal.isna()]
nans = nans1.codmpio
for mpio in nans:
    dpto = geoh[(geoh.codmpio==mpio)].coddpto.iloc[0]
    lista_candidatos = geoh[(geoh.coddpto==dpto) & (geoh.dist_creal.notna())]
    if len(lista_candidatos)>0:
        vecino = encontrar_municipio_mas_cercano(mpio, lista_candidatos.codmpio, geoh)
        vecinos.append(vecino)
    else:
        vecino = None
        vecinos.append(vecino)
    # print(mpio, vecino)
    # vecinos.append(vecino)

nans1["vecino"] = vecinos


def reemplazar_nans_con_vecinos(nans1, notnans1):
    for index, row in nans1.iterrows():
        if not pd.isna(row['vecino']):
            # Buscar la fila del vecino en notnans1
            vecino_info = notnans1[notnans1['codmpio'] == row['vecino']].iloc[0]
            # Reemplazar NaNs
            for col in ['dist_creal', 'dmagdalena', 'dcauca', 'dcosta']:
                if pd.isna(row[col]):
                    nans1.at[index, col] = vecino_info[col]

    return nans1

# Ejecutar la función
nans1_actualizado = reemplazar_nans_con_vecinos(nans1, notnans1)

In [123]:
geohistf = pd.concat([notnans1, nans1_actualizado], axis = 0)
geohistf = geohistf.drop(columns = "vecino")

In [124]:
def imputar_percentil(geohistf):
    for column in geohistf.columns:
        if geohistf[column].isna().any():
            # Calcular el percentil 76 para la columna actual
            percentile_76 = np.percentile(geohistf[column].dropna(), 50)
            # Imputar este percentil donde haya NaNs
            geohistf[column].fillna(percentile_76, inplace=True)
    return geohistf

# Ejecutar la función
geohistf_actualizado = imputar_percentil(geohistf)

## Merge

In [126]:
df  = pd.merge(ed,cg, "outer", ["codmpio", "date"])
df  = pd.merge(df,bg, "outer", ["codmpio", "date"])

In [127]:
df = df.merge(geohistf_actualizado, "left", "codmpio")

In [128]:
df.to_csv("controles.csv", index = False)